## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Vardo,NO,70.3705,31.1107,37.38,87,75,9.22,broken clouds
1,Georgetown,MY,5.4112,100.3354,82.33,93,40,4.61,scattered clouds
2,Gornopravdinsk,RU,60.0500,69.9000,34.25,85,100,7.56,overcast clouds
3,Atuona,PF,-9.8000,-139.0333,79.00,71,29,18.19,scattered clouds
4,Barra Do Garcas,BR,-15.8900,-52.2567,90.23,45,99,5.53,overcast clouds


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature, in Farenheit, that you would like for your trip?"))
max_temp = float(input("What is the maximum temperature, in Farenheit, that you would like for your trip?"))

What is the minimum temperature, in Farenheit, that you would like for your trip?70
What is the maximum temperature, in Farenheit, that you would like for your trip?85


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & 
                                      (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,Georgetown,MY,5.4112,100.3354,82.33,93,40,4.61,scattered clouds
3,Atuona,PF,-9.8000,-139.0333,79.00,71,29,18.19,scattered clouds
5,Sao Filipe,CV,14.8961,-24.4956,75.56,70,8,4.45,clear sky
6,Broome,US,42.2506,-75.8330,73.53,49,74,5.01,broken clouds
7,Rio Grande,BR,-32.0350,-52.0986,78.91,49,1,16.06,clear sky


In [13]:
# 4a. Determine if there are any empty rows.
print(preferred_cities_df.count())

City                   262
Country                260
Lat                    262
Lng                    262
Max Temp               262
Humidity               262
Cloudiness             262
Wind Speed             262
Current Description    262
dtype: int64


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
print(preferred_cities_df.count())

City                   260
Country                260
Lat                    260
Lng                    260
Max Temp               260
Humidity               260
Cloudiness             260
Wind Speed             260
Current Description    260
dtype: int64


In [35]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Current Description','Lat', 'Lng']].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,82.33,scattered clouds,5.4112,100.3354,
3,Atuona,PF,79.00,scattered clouds,-9.8000,-139.0333,
5,Sao Filipe,CV,75.56,clear sky,14.8961,-24.4956,
6,Broome,US,73.53,broken clouds,42.2506,-75.8330,
7,Rio Grande,BR,78.91,clear sky,-32.0350,-52.0986,
8,Kahului,US,74.84,scattered clouds,20.8947,-156.4700,
10,Itarema,BR,83.32,scattered clouds,-2.9248,-39.9167,
11,Jamestown,US,80.02,few clouds,42.0970,-79.2353,
15,Mar Del Plata,AR,76.95,clear sky,-38.0023,-57.5575,
17,Sao Gabriel Da Cachoeira,BR,78.60,light rain,-0.1303,-67.0892,


In [36]:
# IMPORT NUMPY
import numpy as np
# 6a. Set parameters to search for hotels within 5000 meters.
params = {'radius': 5000, 'type': 'lodging', 'key': g_key}

# 6b. Iterate through the DataFrame

for index, row in hotel_df.iterrows():
    # Get the lat and long
    lat = row['Lat']
    lng = row['Lng']
    
    # 6c. Add the lat and long to location key for the params dict
    params['location'] = f"{lat},{lng}"
    
    # 6d. Use the search term: 'lodging' and our lat and long
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Grab the first hotel from the results and store the name, give nulls if no hotel found
    city_hotel = hotel_df.loc[index, "City"]
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print(f"Hotel not found for {city_hotel} ...skipping.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found for Avera ...skipping.
Hotel not found for Lagos ...skipping.
Hotel not found for Gazli ...skipping.
Hotel not found for Marzuq ...skipping.
Hotel not found for Poya ...skipping.
Hotel not found for Thinadhoo ...skipping.
Hotel not found for Abu Samrah ...skipping.
Hotel not found for Amapa ...skipping.
Hotel not found for Obo ...skipping.
Hotel not found for Ligayan ...skipping.
Hotel not found for Beloha ...skipping.
Hotel not found for Bulungu ...skipping.
Hotel not found for Merauke ...skipping.
Hotel not found for Luebo ...skipping.
Hotel not found for Lodja ...skipping.
Hotel not found for Mtambile ...skipping.


In [37]:
# Checking to see if it added hotel names properly
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Georgetown,MY,82.33,scattered clouds,5.4112,100.3354,Cititel Penang
3,Atuona,PF,79.00,scattered clouds,-9.8000,-139.0333,Villa Enata
5,Sao Filipe,CV,75.56,clear sky,14.8961,-24.4956,Hotel Xaguate
6,Broome,US,73.53,broken clouds,42.2506,-75.8330,Chenango Valley State Park
7,Rio Grande,BR,78.91,clear sky,-32.0350,-52.0986,Hotel Atlântico Rio Grande


In [38]:
# count nulls and check
print(hotel_df.count())

City                   260
Country                260
Max Temp               260
Current Description    260
Lat                    260
Lng                    260
Hotel Name             244
dtype: int64


In [39]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [40]:
# Check to see if properly dropped (should be 244 across the board)
print(hotel_df.count())

City                   244
Country                244
Max Temp               244
Current Description    244
Lat                    244
Lng                    244
Hotel Name             244
dtype: int64


In [41]:
# 8a. Create the output File (CSV)
# 8b. Export the City_Data (the hotel_df) into a csv
hotel_df.to_csv("WeatherPy_Vacation.csv", index_label="City_ID")

In [42]:
# 9. Using the template add hotel name, city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>

<dt>Hotel</dt><dd>{Hotel Name}</dd>

<dt>City</dt><dd>{City}</dd>

<dt>Country</dt><dd>{Country}</dd>

<dt>Current Weather</dt><dd>{Current Description} at {Max Temp} °F</dd>

</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations_df = hotel_df[["Lat", "Lng"]]

In [1]:
# Add a heatmap of temperature for the vacation spots
locations = hotel_df[['Lat','Lng']]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center =(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

# 11a. Add a marker layer for each city to the map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Call the figure
fig

NameError: name 'hotel_df' is not defined